In [0]:
#creating a dataframe for Q.1
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,LongType
def createDF():
        schema = StructType([
            StructField("Product",StringType(), True),
            StructField("Amount", IntegerType(), True),
            StructField("Country", StringType(), True),
            ])
        data = [("Banana", 1000,"USA"),
            ("Carrots",1500,"India"),
             ("Beans",1600 , "Sweden"),
            ("Orange",2000,"UK"),
            ("Orange",2000,"UAE"),
            ("Banana",400,"China"),
            ("Carrots",1200,"China")]
        df = spark.createDataFrame(data, schema)
        return df
df = createDF()

df.createOrReplaceTempView("table")
pivot_df = spark.sql("SELECT product, USA, India, Sweden, UK, UAE, China \
                     FROM (SELECT product, amount, country FROM table) \
                     PIVOT (SUM(amount) FOR country IN ('USA', 'India', 'Sweden', 'UK', 'UAE', 'China'))")
display(pivot_df)

pivot_df.createOrReplaceTempView("table")
unpivot_df = spark.sql("""
    SELECT Product, 'USA' AS Country, USA AS Amount FROM table
    UNION ALL
    SELECT Product, 'India' AS Country, India AS Amount FROM table
    UNION ALL
    SELECT Product, 'Sweden' AS Country, Sweden AS Amount FROM table
    UNION ALL
    SELECT Product, 'UK' AS Country, UK AS Amount FROM table
    UNION ALL
    SELECT Product, 'UAE' AS Country, UAE AS Amount FROM table
    UNION ALL
    SELECT Product, 'China' AS Country, China AS Amount FROM table
    WHERE USA IS NOT NULL OR India IS NOT NULL OR Sweden IS NOT NULL OR UK IS NOT NULL OR UAE IS NOT NULL OR China IS NOT NULL
""")

display(unpivot_df)

             


product,USA,India,Sweden,UK,UAE,China
Orange,null,null,null,2000,2000,null
Beans,null,null,1600,null,null,null
Banana,1000,null,null,null,null,400
Carrots,null,1500,null,null,null,1200


Product,Country,Amount
Orange,USA,null
Beans,USA,null
Banana,USA,1000
Carrots,USA,null
Orange,India,null
Beans,India,null
Banana,India,null
Carrots,India,1500
Orange,Sweden,null
Beans,Sweden,1600
